In this file can be found different methods used to predict the price, and those can be used to replace the algorithm of linear regression in the file "algo.ipynb".

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import matplotlib.pyplot as plt
#from lightgbm import LGBMRegressor
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns

In [2]:
#importing the dataset
import pandas as pd
df = pd.read_csv('/home/jehona/PythonDataScienceHandbook-master/renfe.csv',sep=',')

In [3]:
df = df.dropna()

In [4]:
df.drop(["Unnamed: 0"], axis = 1, inplace = True)
df.drop(["insert_date"], axis = 1, inplace = True)

In [5]:
df['start_date'] =pd.to_datetime(df['start_date'])
df['end_date']=pd.to_datetime(df['end_date'])

In [6]:
# create new columns
import numpy as np
df['duration']=df['end_date']-df['start_date']
df['duration_mins']=df['duration']/np.timedelta64(1, 'm')
df['start_time']=df['start_date'].dt.time

In [7]:
# add a column with duration
import datetime as dt
df['duration (h)'] = (df['end_date']-df['start_date']).dt.seconds/3600

In [8]:
# add a column with trip distance - it will allow us to differentiate routes
# because there are only few routes, we can treat this variable as a string, not necessarily as a continuous one

# create a dictionary with stations cordinates
coordinates = {'MADRID':[40.4065,3.6896],\
               'SEVILLA':[37.3911,5.9755],\
               'PONFERRADA':[42.5454,6.6023],\
               'BARCELONA':[41.3790,2.1399],\
               'VALENCIA':[39.4666,0.3773]}

# create columns with origin and destination coordinates
df['origin_coordinates'] = df['origin'].apply(lambda x: coordinates[x])
df['destination_coordinates'] = df['destination'].apply(lambda x: coordinates[x])

# put together a function for calculating distance
def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c
    
    return d

# create the distance function
df['distance (km)'] = df.apply(lambda row: distance(row['origin_coordinates'],row['destination_coordinates']), axis=1)

# get basic statistics for the function
df['distance (km)'].describe()

# remove columns not needed aymore
df = df.drop(columns=['origin_coordinates','destination_coordinates'])

In [9]:
# create a column with origin and destination (this variable is effectively equivalent to the distance one)
df['origin - destination'] = df['origin']+' - '+df['destination']

In [10]:
# create variables to reflect seasonal price movements
df['month'] = df['start_date'].dt.month
df['weekday'] = df['start_date'].dt.weekday
df['day'] = df['start_date'].dt.day
df['hour'] = df['start_date'].dt.hour
df['minute'] = df['start_date'].dt.minute

In [11]:
df.head()

,origin,destination,start_date,end_date,train_type,price,train_class,fare,duration,duration_mins,start_time,duration (h),distance (km),origin - destination,month,weekday,day,hour,minute
0,MADRID,SEVILLA,2019-05-29 06:20:00,2019-05-29 09:16:00,AV City,38.55,Turista,Promo,02:56:00,176.0,06:20:00,2.933333,389.26503,MADRID - SEVILLA,5,2,29,6,20
1,MADRID,SEVILLA,2019-05-29 07:00:00,2019-05-29 09:32:00,AVE,53.40,Turista,Promo,02:32:00,152.0,07:00:00,2.533333,389.26503,MADRID - SEVILLA,5,2,29,7,0
2,MADRID,SEVILLA,2019-05-29 07:30:00,2019-05-29 09:51:00,AVE,47.30,Turista,Promo,02:21:00,141.0,07:30:00,2.350000,389.26503,MADRID - SEVILLA,5,2,29,7,30
3,MADRID,SEVILLA,2019-05-29 08:00:00,2019-05-29 10:32:00,AVE,69.40,Preferente,Promo,02:32:00,152.0,08:00:00,2.533333,389.26503,MADRID - SEVILLA,5,2,29,8,0
5,MADRID,SEVILLA,2019-05-29 09:00:00,2019-05-29 11:38:00,AVE,60.30,Turista,Promo,02:38:00,158.0,09:00:00,2.633333,389.26503,MADRID - SEVILLA,5,2,29,9,0


In [12]:
# target variable
y = df['price']

In [13]:
# features
features = pd.concat([pd.get_dummies(df[['train_type','train_class','fare','month','weekday','day','hour','minute','duration (h)']]),\
           pd.get_dummies(df[['distance (km)']].astype(str)),\
                     ],axis=1)

In [14]:
#one way of preparing the data is using One Hot Encoder, which transforms the data from categorical to numeric
from sklearn.preprocessing import OneHotEncoder

onehotencoder = OneHotEncoder()
X = onehotencoder.fit_transform(features.values)

/home/jehona/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [15]:
# split into train and test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [19]:
#SVM
from sklearn.svm import LinearSVR
svm = LinearSVR()
svm.fit(X_train, y_train)


LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0)

In [20]:
train_score_svm = svm.score(X_train, y_train)
test_score_svm = svm.score(X_test, y_test)

print("Train Score:", train_score_svm)
print("Test Score:", test_score_svm)
print()

from sklearn.metrics import mean_squared_error
def compute_mse(model, X, y_true, name):
    y_pred = svm.predict(X)
    mse = mean_squared_error(y_true, y_pred)
    print(f'Mean Squared Error for {name}: {mse}')

compute_mse(svm, X_train, Y_train, 'training set')
compute_mse(svm, X_test, Y_test, 'test set')

Train Score: 0.83941029497407
Test Score: 0.8389978473033983



NameError: name 'Y_train' is not defined

Train score with SVR: 0.839
Tet score: 0.838


In [17]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [18]:
train_score_lr = model.score(X_train, y_train)
test_score_lr = model.score(X_test, y_test)
print("Train Score:", train_score_lr)
print("Test Score:", test_score_lr)
print()

Train Score: 0.8538230592781568
Test Score: 0.8539768967969089



Train score with linear regression: 0.8538
Test score: 0.8539

In [24]:
from sklearn.ensemble import RandomForestRegressor
# Instantiate a RandomRegressor object
MAXDEPTH = 20
rf_model = RandomForestRegressor(n_estimators=1200,   # No of trees in forest
                             criterion = "mse",       # Can also be mae
                             max_features = "sqrt",   # no of features to consider for the best split
                             max_depth= MAXDEPTH,     #  maximum depth of the tree
                             min_samples_split= 2,    # minimum number of samples required to split an internal node
                             min_impurity_decrease=0, # Split node if impurity decreases greater than this value.
                             oob_score = True,        # whether to use out-of-bag samples to estimate error on unseen data.
                             n_jobs = -1,             #  No of jobs to run in parallel
                             random_state=0,
                             verbose = 10             # Controls verbosity of process
                             )

rf_model.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 1200building tree 2 of 1200building tree 3 of 1200
building tree 4 of 1200


building tree 5 of 1200
building tree 6 of 1200
building tree 7 of 1200
building tree 8 of 1200
building tree 9 of 1200


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.9min


building tree 10 of 1200
building tree 11 of 1200
building tree 12 of 1200
building tree 13 of 1200


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.0min


building tree 14 of 1200
building tree 15 of 1200
building tree 16 of 1200
building tree 17 of 1200
building tree 18 of 1200
building tree 19 of 1200
building tree 20 of 1200


KeyboardInterrupt: 

In [26]:
# gradient boost model with 6000 estimators
from lightgbm import LGBMRegressor
gbr = LGBMRegressor(n_estimators=6000)
gbr.fit(X_train, y_train)
gbr.score(X_train, y_train)
gbr.score(X_test, y_test)


0.9809972017923134